In [128]:
# Hyperparameters
POPULATION_SIZE = 8
MAXIMUM_WEIGHT = 450    #529

### GA的順序
evalutation $\rightarrow$ selection $\rightarrow$ crossover $\rightarrow$ mutation
### 要注意的地方
1. 除了 Equipment, 每種 Type 只能選一個 Item.
2. Maximum weight 529 units. (後暫時更改為450)
3. At least one knife, one pistol and one equipment in inventory bag.
4. 指定GA內部operators - 使用 roulette-wheel selection, uniform crossover (p=0.1), multi bit flip
5.

* 下列組合 - extra credits (依 主武器、副武器、小刀 及 裝備 排序）
    * (Leone YG1265, Desert Eagle, - , shield) S.P. +25
    * (Krieg 500, Desert Eagle, - , shield) S.P. +25
    * All three equipments packed (Gas Mask + NV Goggle + Shield) S.P. +70

#### Data Processing

In [129]:
#### read csv files as dataframe
import pandas as pd
import numpy as np

List_of_Gear = pd.read_csv('List_of_gear.csv')
List_of_Gear

,Item,Type,Weight,Survival_Points
0,Shadow_Daggers,Knife,3.3,7
1,Huntsman_Knife,Knife,3.4,8
2,Gut_Knife,Knife,6.0,13
3,228_Compact_Handgun,Pistols,26.1,29
4,Night_Hawk,Pistols,37.6,48
5,Desert_Eagle_Magnum,Pistols,62.5,99
6,Ingram_MAC-10_SMG,Primary,100.2,177
7,Leone_YG1265_Auto_Shotgun,Primary,141.1,213
8,M4A1_Carbine,Primary,119.2,202
9,AK-47_Rifle,Primary,122.4,210


#### Pre-work for later data processing

In [130]:
## Create an Item - Weight mapping
Weight_Map = List_of_Gear.iloc[:,2].to_numpy()
#print(Weight_Map)
## Create an Item - Survival_Points mapping
Survival_Points_Map = List_of_Gear.iloc[:,3].to_numpy()
#print(Survival_Points_Map)

#### Some Methods

In [136]:
import numpy as np

def Check_Overweight(Given_Array):
    Weight = Weight_Mapping(Given_Array)
    if Weight <= MAXIMUM_WEIGHT:
        return 0
    else:
        return 1

def Survival_Points_Mapping(Given_Array):
    Sum_of_Survival_Points = 0
    for i in range(len(Given_Array)):
        if Given_Array[i] == 1:
            Sum_of_Survival_Points += Survival_Points_Map[i]
    # Extra Points
        ## YG1265 or Krieg 500 + Desert Eagle + tactical shield, survival points +25
    if(((Given_Array[7]==1)|(Given_Array[10]==1))&(Given_Array[5]==Given_Array[14]==1)):
        Sum_of_Survival_Points += 25
        ## shadow dagger + desert eagle, survival points +5
    elif(Given_Array[0]==Given_Array[5]==1):
        Sum_of_Survival_Points += 5
        ## 228 Compact + AK-47 or M4A1, survival points +15
    elif(((Given_Array[8]==1)|(Given_Array[9]==1))&(Given_Array[4]==1)):
        Sum_of_Survival_Points += 15
    elif(Given_Array[12]==Given_Array[13]==Given_Array[14]==1):
        Sum_of_Survival_Points += 70
    else:
        pass
    return Sum_of_Survival_Points

def Weight_Mapping(Given_Array):
    Sum_of_weight = 0
    for i in range(len(Given_Array)):
        if Given_Array[i] == 1:
            Sum_of_weight += Weight_Map[i]
    return Sum_of_weight

def Randomly_Select_A_Row_from_Matrix(Given_Matrix):
    number_of_rows = Given_Matrix.shape[0]
    random_indicies = np.random.choice(number_of_rows, size=1, replace=False)
    Selected_Row = Given_Matrix[random_indicies, :]
    Selected_Row = np.squeeze(Selected_Row)
    return Selected_Row

def Generate_Single_Creature():
# Except equipments, we can only choose one item in each type.
# I will set the initial choice of each items by utilizing np.eye matrix,\
# \then randomly select a row in order to represent the collection of items chosen.
    Knife = np.eye(3); Pistols = np.eye(3); Primary = np.eye(6);
    Selected_Knife = Randomly_Select_A_Row_from_Matrix(Knife)
    Selected_Pistols = Randomly_Select_A_Row_from_Matrix(Pistols)
    Selected_Primary = Randomly_Select_A_Row_from_Matrix(Primary)
    Equipments = np.random.randint(2, size=3)
    Items_in_Bag = np.concatenate((Selected_Knife, Selected_Pistols, Selected_Primary, Equipments))

    if Check_Overweight(Items_in_Bag)==0:
        return Items_in_Bag
    else:
        return Generate_Single_Creature()   #Recursive Call till no overweight

def Generate_Population():
    Population = []
    for i  in range(POPULATION_SIZE):
        Single_Creature = Generate_Single_Creature()
        #Single_Creature = np.array(Single_Creature)
        Population.append(Single_Creature)
    print(f'Population successfully created.')
    print(f'Population shape:{np.array(Population).shape}')
    return Population

def Roulete_Wheel_Selection(Given_Matrix):

    A = []
    max = -np.inf
    for i in range(len(Population)):
        survial_point = Survival_Points_Mapping(Population[i])
        A.append(survial_point)
        if survial_point > max:
            max = survial_point
    print(f'A:{A}')
    print(f'max:{max}')

    sum = 0
    for j in range(len(Population)):
        #sum = sum + np.max([1,max-A[j]])
        sum += A[j]
        A[j] = sum
    print(A)

    Matepool = []
    for k in range(int(len(Population)/2)):
        a = np.random.randint(sum)
        #print(f'a:{a}')
        i=0
        while(a>A[i]):
            i += 1
        Matepool.append(Population[i])

    print(f'Matepool:\n{Matepool}')
    print(f'shape of Matepool: {np.array(Matepool).shape}')

    return 0

Population = Generate_Population()
a = Roulete_Wheel_Selection(Population)
print(f'Population: {Population}')
print(f'len(Population): {len(Population)}')

# 檢查 Popultaion 中，第一個 creatrue 的資訊
print(f'Population[0]: {Population[0]}')
x = Weight_Mapping(Population[0])
print(x)

Population successfully created.
Population shape:(8, 15)
A:[247, 359, 374, 257, 315, 226, 272, 277]
max:374
[247, 606, 980, 1237, 1552, 1778, 2050, 2327]
Matepool:
[array([1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.]), array([0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1.]), array([1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.]), array([0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.])]
shape of Matepool: (4, 15)
Population: [array([1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]), array([1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.]), array([0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1.]), array([0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1.]), array([1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.]), array([0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]), array([1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), array([1., 0., 0., 0., 1., 0., 0., 

#### Let's Start

#### Test

In [132]:
import numpy as np
x = -np.inf
print(x)

y = np.random.randint(100)
print(y)

A = [1,1,1]

if (A[0]==A[1]==1):
    print(2)

B = np.array([12, 23, 34, 45])
C = B-15
print(C)

-inf
49
2
[-3  8 19 30]
